In [1]:
import os
import requests
from bs4 import BeautifulSoup
import sqlite3
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError, LineBotApiError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage, ButtonsTemplate, PostbackTemplateAction, PostbackEvent

app = Flask(__name__)

# 設定 LINE Bot 的 Channel Access Token 和 Channel Secret
LINE_CHANNEL_ACCESS_TOKEN = os.getenv('LINE_ACCESS_TOKEN')
LINE_CHANNEL_SECRET = os.getenv('LINE_SECRET')

line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

# 全域變數，用於暫存所選擇的商品資訊
selected_product_info = {}
awaiting_custom_query = False  # 用於標記是否在等待用戶輸入自定義查詢
selected_store = None  # 用於存儲所選擇的商店

# 設定Flask路由
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature', '')
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 設計按鈕模板消息
def create_store_buttons():
    buttons_template = TemplateSendMessage(
        alt_text='商店選單',
        template=ButtonsTemplate(
            title='請選擇商店',
            text='請選擇您想要的商店',
            actions=[
                PostbackTemplateAction(
                    label='傑昇通信',
                    data='action=choose_store&store=傑昇通信'
                ),
                PostbackTemplateAction(
                    label='地標網通',
                    data='action=choose_store&store=地標網通'
                ),
                PostbackTemplateAction(
                    label='皆可',
                    data='action=choose_store&store=皆可'
                )
            ]
        )
    )
    return buttons_template

def create_phone_buttons():
    buttons_template = TemplateSendMessage(
        alt_text='手機選單',
        template=ButtonsTemplate(
            title='請選擇手機系統',
            text='請選擇您想要的手機系統',
            actions=[
                PostbackTemplateAction(
                    label='Android',
                    data='action=choose&system=android'
                ),
                PostbackTemplateAction(
                    label='iOS',
                    data='action=choose&system=ios'
                )
            ]
        )
    )
    return buttons_template

# 處理文字訊息事件
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global awaiting_custom_query  # 使用全域變數

    if event.message.text == "手機":
        buttons_template = create_store_buttons()
        line_bot_api.reply_message(event.reply_token, buttons_template)
    elif event.message.text == "3C新聞":
        latest_info = get_latest_article()
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=latest_info))
    elif awaiting_custom_query:  # 如果等待用戶輸入自定義查詢
        query = event.message.text
        if query == "結束":
            awaiting_custom_query = False
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='搜尋結束。'))
        else:
            send_custom_query_result(event.reply_token, query)

# 處理Postback事件
@handler.add(PostbackEvent)
def handle_postback(event):
    global awaiting_custom_query, selected_store  # 使用全域變數

    data = event.postback.data
    if data.startswith('action=choose_store&store='):
        selected_store = data.split('=')[-1]
        buttons_template = create_phone_buttons()
        line_bot_api.reply_message(event.reply_token, buttons_template)
    elif data.startswith('action=choose&system='):
        system = data.split('=')[-1]
        if system == 'android' or system == 'ios':
            send_brand_buttons(event.reply_token, system)
    elif data.startswith('action=choose_brand&brand='):
        brand = data.split('=')[-1]
        if brand == '其他':
            awaiting_custom_query = True  # 設置標記等待用戶輸入自定義查詢
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='請輸入您想要搜尋的產品名稱，或輸入"結束"來結束搜尋：\n範例: HTC U12\niPhone 15\nSamsung A55'))
        else:
            send_model_buttons(event.reply_token, brand)
    elif data.startswith('action=choose_model&model='):
        model = data.split('=')[-1]
        if model == '其他':
            awaiting_custom_query = True  # 設置標記等待用戶輸入自定義查詢
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='請輸入您想要搜尋的產品名稱，或輸入"結束"來結束搜尋：\n範例: HTC U12\niPhone 15\nSamsung A55'))
        else:
            send_product_info(event.reply_token, model)

# 設計品牌選單
def send_brand_buttons(reply_token, system):
    if system == 'ios':
        brands = ['Apple']
    else:
        brands = ['SAMSUNG', 'ASUS', 'SONY', '其他']

    actions = [PostbackTemplateAction(label=brand, data=f'action=choose_brand&brand={brand}') for brand in brands]

    buttons_template = TemplateSendMessage(
        alt_text='手機品牌選單',
        template=ButtonsTemplate(
            title=f'請選擇{system.capitalize()}手機品牌',
            text=f'請選擇您想要的{system.capitalize()}手機品牌',
            actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, buttons_template)

# 設計型號選單
def send_model_buttons(reply_token, brand):
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    query = "SELECT name FROM products WHERE name LIKE ?"
    params = (f'{brand}%',)

    if selected_store != '皆可':
        query += " AND store = ?"
        params = (f'{brand}%', selected_store)

    c.execute(query, params)
    models = [row[0] for row in c.fetchall()]
    conn.close()

    actions = []
    for model in models[:3]:  # 確保前三個選項
        label = model if len(model) <= 20 else model[:20]  # 確保標籤長度不超過20個字元
        actions.append(PostbackTemplateAction(label=label, data=f'action=choose_model&model={model}'))

    actions.append(PostbackTemplateAction(label='其他', data='action=choose_model&model=其他'))  # 添加“其他”選項

    buttons_template = TemplateSendMessage(
        alt_text='手機型號選單',
        template=ButtonsTemplate(
            title=f'請選擇{brand}手機型號',
            text=f'請選擇您想要的{brand}手機型號',
            actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, buttons_template)

# 查詢並發送自定義產品結果
def send_custom_query_result(reply_token, query):
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    sql_query = "SELECT name, price, store FROM products WHERE name LIKE ?"
    params = (f'%{query}%',)

    if selected_store != '皆可':
        sql_query += " AND store = ?"
        params = (f'%{query}%', selected_store)

    c.execute(sql_query, params)
    rows = c.fetchall()
    conn.close()

    if rows:
        messages = [TextSendMessage(text=f'商品名稱: {row[0]}, 價格: {row[1]}, 商店: {row[2]}') for row in rows]
    else:
        messages = [TextSendMessage(text='找不到相關商品')]

    # 確保一次回覆的消息數量不超過 5
    if len(messages) > 4:
        line_bot_api.reply_message(reply_token, messages[:4])
    else:
        line_bot_api.reply_message(reply_token, messages)

    # 提示用戶繼續輸入
    line_bot_api.push_message(event.source.user_id, TextSendMessage(text='請輸入您想要搜尋的產品名稱，或輸入"結束"來結束搜尋：'))

# 從資料庫中檢索商品資訊
def send_product_info(reply_token, model):
    global selected_product_info  # 使用全域變數

    if model in selected_product_info:  # 如果已經暫存了該商品資訊，直接回覆
        product_info = selected_product_info[model]
        line_bot_api.reply_message(reply_token, product_info)
    else:  # 否則從資料庫中查詢並回覆
        conn = sqlite3.connect('products.db')
        c = conn.cursor()
        sql_query = 'SELECT name, price, store FROM products WHERE name = ?'
        params = (model,)

        if selected_store != '皆可':
            sql_query += " AND store = ?"
            params = (model, selected_store)

        c.execute(sql_query, params)
        rows = c.fetchall()
        conn.close()

        if rows:
            messages = [TextSendMessage(text=f'商品名稱: {row[0]}, 價格: {row[1]}, 商店: {row[2]}') for row in rows]
            selected_product_info[model] = messages  # 暫存商品資訊
        else:
            messages = [TextSendMessage(text='找不到相關商品')]

        line_bot_api.reply_message(reply_token, messages)

# 獲取最新文章
def get_latest_article():
    url = 'https://ccc.technews.tw/'
    response = requests.get(url)
    if response.status_code != 200:
        return "無法存取該網址。"
    else:
        soup = BeautifulSoup(response.text, 'html.parser')
        latest_article = soup.find('div', class_='content')
        if latest_article:
            title = latest_article.find('h1', class_='entry-title').text.strip()
            link = latest_article.find('a')['href']
            return f"{title}\n{link}"
        else:
            return "找不到最新文章。"

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\USER\AppData\Local\Temp\ipykernel_24620\2406314608.py:16: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
C:\Users\USER\AppData\Local\Temp\ipykernel_24620\2406314608.py:17: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(LINE_CHANNEL_SECRET)
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [11]:
import os
import requests
from bs4 import BeautifulSoup
import sqlite3
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError, LineBotApiError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage, ButtonsTemplate, PostbackTemplateAction, PostbackEvent
import random

app = Flask(__name__)

# 設定 LINE Bot 的 Channel Access Token 和 Channel Secret
LINE_CHANNEL_ACCESS_TOKEN = os.getenv('LINE_ACCESS_TOKEN')
LINE_CHANNEL_SECRET = os.getenv('LINE_SECRET')

line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

# 全域變數，用於暫存所選擇的商品資訊
selected_product_info = {}
awaiting_custom_query = False  # 用於標記是否在等待用戶輸入自定義查詢
selected_store = None  # 用於存儲所選擇的商店
current_function = None  # 用於標記當前功能（手機、新聞、筆電）

# 設定Flask路由
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature', '')
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 設計按鈕模板消息
def create_store_buttons():
    buttons_template = TemplateSendMessage(
        alt_text='商店選單',
        template=ButtonsTemplate(
            title='請選擇商店',
            text='請選擇您想要的商店',
            actions=[
                PostbackTemplateAction(
                    label='創捷國際',
                    data='action=choose_store&store=創捷國際'
                ),
                PostbackTemplateAction(
                    label='台灣大哥大',
                    data='action=choose_store&store=台灣大哥大'
                ),
                PostbackTemplateAction(
                    label='皆可',
                    data='action=choose_store&store=皆可'
                )
            ]
        )
    )
    return buttons_template

def create_system_buttons():
    buttons_template = TemplateSendMessage(
        alt_text='系統選單',
        template=ButtonsTemplate(
            title='請選擇系統',
            text='請選擇您想要的系統',
            actions=[
                PostbackTemplateAction(
                    label='Mac os',
                    data='action=choose_system&system=mac'
                ),
                PostbackTemplateAction(
                    label='Windows',
                    data='action=choose_system&system=windows'
                )
            ]
        )
    )
    return buttons_template

def create_phone_buttons():
    buttons_template = TemplateSendMessage(
        alt_text='手機選單',
        template=ButtonsTemplate(
            title='請選擇手機系統',
            text='請選擇您想要的手機系統',
            actions=[
                PostbackTemplateAction(
                    label='Android',
                    data='action=choose&system=android'
                ),
                PostbackTemplateAction(
                    label='iOS',
                    data='action=choose&system=ios'
                )
            ]
        )
    )
    return buttons_template

# 處理文字訊息事件
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global awaiting_custom_query, current_function  # 使用全域變數

    if event.message.text == "筆電":
        current_function = "筆電"
        buttons_template = create_store_buttons()
        line_bot_api.reply_message(event.reply_token, buttons_template)
    elif event.message.text == "手機":
        current_function = "手機"
        buttons_template = create_store_buttons()
        line_bot_api.reply_message(event.reply_token, buttons_template)
    elif event.message.text == "3C新聞":
        current_function = "新聞"
        latest_info = get_latest_article()
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=latest_info))
    elif awaiting_custom_query:  # 如果等待用戶輸入自定義查詢
        query = event.message.text
        if query == "結束":
            awaiting_custom_query = False
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='搜尋結束。'))
        else:
            send_custom_query_result(event.reply_token, query)

# 處理Postback事件
@handler.add(PostbackEvent)
def handle_postback(event):
    global awaiting_custom_query, selected_store, current_function  # 使用全域變數

    data = event.postback.data
    if data.startswith('action=choose_store&store='):
        selected_store = data.split('=')[-1]
        if current_function == "筆電":
            buttons_template = create_system_buttons()
        elif current_function == "手機":
            buttons_template = create_phone_buttons()
        line_bot_api.reply_message(event.reply_token, buttons_template)
    elif data.startswith('action=choose_system&system='):
        system = data.split('=')[-1]
        if system == 'mac':
            send_mac_product_buttons(event.reply_token)
        elif system == 'windows':
            if selected_store == '台灣大哥大':
                send_twm_windows_brand_buttons(event.reply_token)
            else:
                send_windows_brand_buttons(event.reply_token)
    elif data.startswith('action=choose&system='):
        system = data.split('=')[-1]
        if system == 'android' or system == 'ios':
            send_brand_buttons(event.reply_token, system)
    elif data.startswith('action=choose_brand&brand='):
        brand = data.split('=')[-1]
        if brand == '其他':
            awaiting_custom_query = True  # 設置標記等待用戶輸入自定義查詢
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='請輸入您想要搜尋的產品名稱，或輸入"結束"來結束搜尋：\n範例: HTC U12\niPhone 15\nSamsung A55'))
        else:
            send_model_buttons(event.reply_token, brand)
    elif data.startswith('action=choose_model&model='):
        model = data.split('=')[-1]
        if model == '其他':
            awaiting_custom_query = True  # 設置標記等待用戶輸入自定義查詢
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='請輸入您想要搜尋的產品名稱，或輸入"結束"來結束搜尋：\n範例: HTC U12\niPhone 15\nSamsung A55'))
        else:
            send_product_info(event.reply_token, model)
    elif data.startswith('action=choose_mac_product&product='):
        product = data.split('=')[-1]
        if product == '其他':
            awaiting_custom_query = True  # 設置標記等待用戶輸入自定義查詢
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='請輸入您想要搜尋的產品名稱，或輸入"結束"來結束搜尋：'))
        else:
            send_product_info(event.reply_token, product)
    elif data.startswith('action=choose_windows_brand&brand='):
        brand = data.split('=')[-1]
        if brand == '其他':
            awaiting_custom_query = True  # 設置標記等待用戶輸入自定義查詢
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='請輸入您想要搜尋的產品名稱，或輸入"結束"來結束搜尋：'))
        else:
            send_windows_model_buttons(event.reply_token, brand)
    elif data.startswith('action=choose_windows_model&model='):
        model = data.split('=')[-1]
        if model == '其他':
            awaiting_custom_query = True  # 設置標記等待用戶輸入自定義查詢
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='請輸入您想要搜尋的產品名稱，或輸入"結束"來結束搜尋：'))
        else:
            send_product_info(event.reply_token, model)

# 設計品牌選單
def send_brand_buttons(reply_token, system):
    if system == 'ios':
        brands = ['Apple']
    else:
        brands = ['SAMSUNG', 'ASUS', 'SONY', '其他']

    actions = [PostbackTemplateAction(label=brand, data=f'action=choose_brand&brand={brand}') for brand in brands]

    buttons_template = TemplateSendMessage(
        alt_text='手機品牌選單',
        template=ButtonsTemplate(
            title=f'請選擇{system.capitalize()}手機品牌',
            text=f'請選擇您想要的{system.capitalize()}手機品牌',
            actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, buttons_template)

# 設計型號選單
def send_model_buttons(reply_token, brand):
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    query = "SELECT name FROM products WHERE name LIKE ?"
    params = (f'{brand}%',)

    if selected_store != '皆可':
        query += " AND store = ?"
        params = (f'{brand}%', selected_store)
    c.execute(query, params)
    rows = c.fetchall()
    conn.close()

    models = [row[0] for row in rows]
    if not models:
        line_bot_api.reply_message(reply_token, TextSendMessage(text='未找到相應型號'))
        return

    actions = [PostbackTemplateAction(label=model, data=f'action=choose_model&model={model}') for model in models[:3]]
    actions.append(PostbackTemplateAction(label='其他', data='action=choose_model&model=其他'))

    buttons_template = TemplateSendMessage(
        alt_text='手機型號選單',
        template=ButtonsTemplate(
            title=f'請選擇{brand}手機型號',
            text=f'請選擇您想要的{brand}手機型號',
            actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, buttons_template)

# 查詢產品信息並回傳給用戶
def send_product_info(reply_token, product_name):
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    query = "SELECT * FROM products WHERE name = ?"
    params = (product_name,)

    if selected_store != '皆可':
        query += " AND store = ?"
        params = (product_name, selected_store)
    c.execute(query, params)
    product_info = c.fetchone()
    conn.close()

    if product_info:
        name, store, price, url = product_info
        message = f"商品名稱: {name}\n商店: {store}\n價格: {price}\n網址: {url}"
    else:
        message = '未找到該商品的信息'

    line_bot_api.reply_message(reply_token, TextSendMessage(text=message))

# 設計 Mac 產品選單
def send_mac_product_buttons(reply_token):
    products = ['MacBook Pro 14', 'MacBook Pro 16', 'MacBook Air 13', '其他']
    actions = [PostbackTemplateAction(label=product, data=f'action=choose_mac_product&product={product}') for product in products]

    buttons_template = TemplateSendMessage(
        alt_text='Mac 筆電選單',
        template=ButtonsTemplate(
            title='請選擇 Mac 筆電型號',
            text='請選擇您想要的 Mac 筆電型號',
            actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, buttons_template)

# 設計 Windows 品牌選單
def send_windows_brand_buttons(reply_token):
    brands = ['ACER', 'ASUS', 'MSI', 'Lenovo', '其他']
    actions = [PostbackTemplateAction(label=brand, data=f'action=choose_windows_brand&brand={brand}') for brand in brands]

    buttons_template = TemplateSendMessage(
        alt_text='Windows 筆電品牌選單',
        template=ButtonsTemplate(
            title='請選擇 Windows 筆電品牌',
            text='請選擇您想要的 Windows 筆電品牌',
            actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, buttons_template)

# 設計台灣大哥大 Windows 品牌選單
def send_twm_windows_brand_buttons(reply_token):
    brands = ['ACER', 'ASUS', '其他']
    actions = [PostbackTemplateAction(label=brand, data=f'action=choose_windows_brand&brand={brand}') for brand in brands]

    buttons_template = TemplateSendMessage(
        alt_text='台灣大哥大 Windows 筆電品牌選單',
        template=ButtonsTemplate(
            title='請選擇 Windows 筆電品牌',
            text='請選擇您想要的 Windows 筆電品牌',
            actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, buttons_template)

# 設計 Windows 型號選單
def send_windows_model_buttons(reply_token, brand):
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    query = "SELECT name FROM products WHERE name LIKE ?"
    params = (f'{brand}%',)

    if selected_store != '皆可':
        query += " AND store = ?"
        params = (f'{brand}%', selected_store)
    c.execute(query, params)
    rows = c.fetchall()
    conn.close()

    models = [row[0] for row in rows]
    if not models:
        line_bot_api.reply_message(reply_token, TextSendMessage(text='未找到相應型號'))
        return

    actions = [PostbackTemplateAction(label=model, data=f'action=choose_windows_model&model={model}') for model in models[:3]]
    actions.append(PostbackTemplateAction(label='其他', data='action=choose_windows_model&model=其他'))

    buttons_template = TemplateSendMessage(
        alt_text='Windows 筆電型號選單',
        template=ButtonsTemplate(
            title=f'請選擇{brand}筆電型號',
            text=f'請選擇您想要的{brand}筆電型號',
            actions=actions
        )
    )
    line_bot_api.reply_message(reply_token, buttons_template)

# 自定義查詢
def send_custom_query_result(reply_token, query):
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    query_str = "SELECT * FROM products WHERE name LIKE ?"
    params = (f'%{query}%',)

    if selected_store != '皆可':
        query_str += " AND store = ?"
        params = (f'%{query}%', selected_store)
    c.execute(query_str, params)
    product_info = c.fetchone()
    conn.close()

    if product_info:
        name, store, price, url = product_info
        message = f"商品名稱: {name}\n商店: {store}\n價格: {price}\n網址: {url}"
    else:
        message = '未找到該商品的信息'

    line_bot_api.reply_message(reply_token, TextSendMessage(text=message))

# 查詢最新文章
def get_latest_article():
    url = "https://www.sogi.com.tw/articles"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    article = soup.find("a", class_="d-block")
    title = article.text.strip()
    link = article["href"]
    return f"最新的 3C 文章是：\n{title}\n詳情請見：{link}"

if __name__ == "__main__":
    app.run(debug=False)




 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\USER\AppData\Local\Temp\ipykernel_11004\753015702.py:17: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
C:\Users\USER\AppData\Local\Temp\ipykernel_11004\753015702.py:18: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(LINE_CHANNEL_SECRET)
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2024-06-03 01:56:12,836] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\envs\linebot\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispat